# Simple LSTM/GRU language modelling

Revised and fixed code from https://github.com/SudaMonster/lstm-language-model.

NB we pack examples sentence wise unlike many other models that do not account for periods.

Possible improvements: pack similar length sentences into batches to reduce the number of padding tokens.

Also note that we tie the input and output weights (https://arxiv.org/abs/1608.05859)

Essentially this is a playground to familiarize oneself with language modelling.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim

from torch.nn.utils import clip_grad_norm_ as clip
from torch.optim.lr_scheduler import StepLR

import const
from data import *
from rnn import *

In [2]:
ptb_datapath_train = 'data/penn/train.txt'
ptb_datapath_valid = 'data/penn/valid.txt'
ptb_datapath_test = 'data/penn/test.txt'

batch_size = 128

ptb_train = DataSet(ptb_datapath_train, batch_size, display_freq=0, max_len=90, trunc_len=90)
ptb_valid = DataSet(ptb_datapath_valid, batch_size, display_freq=0, max_len=90, trunc_len=90)
ptb_test = DataSet(ptb_datapath_test, batch_size, display_freq=0, max_len=90, trunc_len=90)

Loading data from data/penn/train.txt ...
Loading data from data/penn/valid.txt ...
Loading data from data/penn/test.txt ...


In [3]:
ptb_train.build_dict()
ptb_valid.change_dict(ptb_train.dictionary)
ptb_test.change_dict(ptb_train.dictionary)

Building dictionary...
Done.
Save dictionary at data/penn/train.txt.dict
Index tokens ...
42068 sentences were processed, 0 longer than maximum length,0 were ignored because zero length
Data discription:
Data name : data/penn/train.txt
Number of sentence : 42068
Number of tokens : 887521
Vocabulary size : 10000
Number of batches : 328
Batch size : 128
Done.
Index tokens ...
3370 sentences were processed, 0 longer than maximum length,0 were ignored because zero length
Data discription:
Data name : data/penn/valid.txt
Number of sentence : 3370
Number of tokens : 70390
Vocabulary size : 10000
Number of batches : 26
Batch size : 128
Done.
Index tokens ...
3761 sentences were processed, 0 longer than maximum length,0 were ignored because zero length
Data discription:
Data name : data/penn/test.txt
Number of sentence : 3761
Number of tokens : 78669
Vocabulary size : 10000
Number of batches : 29
Batch size : 128
Done.


In [4]:
# Plot sentence length histogram
# ss = []
# for s in ptb_train.sentence:
#     ss.append(len(s))
# ss = np.array(ss)

# hist = np.histogram(ss)
# plt.hist(ss, bins=100)

In [5]:
rnn_type = 'LSTM'
voc_size = ptb_train.num_vocb
emb_dim = 512
hid_dim = 512
n_layers = 3

lr = 1e-3
n_epochs = 30
clip_grad = 5

In [6]:
model = rnn_model(rnn_type, voc_size, emb_dim, hid_dim, n_layers, tie_weights=True)
criterion = nn.CrossEntropyLoss(ignore_index=const.PAD)

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

opt = optim.Adam(model.parameters(), lr=lr)
# lr_lambda = lambda epoch: 0.99 ** epoch
lrsched = StepLR(opt, step_size=10, gamma=0.5)

In [7]:
for epoch in range(n_epochs):
    lrsched.step()
    acc_loss = 0
    print('Start epoch %d, learning rate %f '%(epoch + 1, opt.state_dict()['param_groups'][0]['lr']))
    start_time = time.time()
    model.train()
    ptb_train.shuffle()
    for batch_idx in tqdm(range(ptb_train.num_batch), unit='batches'):
        data, lengths, target = ptb_train.get_batch(batch_idx)
        
        opt.zero_grad()
        output_flat = model.forward(data, lengths)
        loss = criterion(output_flat, target.view(-1))
        loss.backward()
        clip(model.parameters(), clip_grad)
        opt.step()
        acc_loss += loss.item()

    avg_loss = acc_loss / ptb_train.num_batch
    print('Epoch : %d, Batch : %d / %d, Loss : %f, Perplexity : %f, Time : %f' 
          % (epoch + 1, batch_idx, ptb_train.num_batch,
             avg_loss, math.exp(avg_loss),
             time.time() - start_time))

    acc_loss = 0
    model.eval()
    for batch_idx in tqdm(range(ptb_test.num_batch), unit='batches'):
        data, lengths, target = ptb_test[batch_idx]
        output = model.forward(data, lengths)
        loss = criterion(output, target.view(-1))
        acc_loss += loss.item()

    val_loss = acc_loss / ptb_test.num_batch
    print('Validation Loss : %f' % val_loss)
    print('Validation Perplexity : %f' % math.exp(val_loss))

  0%|          | 1/328 [00:00<00:45,  7.20batches/s]

Start epoch 1, learning rate 0.001000 
2


 10%|█         | 3/29 [00:00<00:00, 29.04batches/s]

Epoch : 1, Batch : 327 / 328, Loss : 6.395267, Perplexity : 599.003170, Time : 30.777261


  0%|          | 0/328 [00:00<?, ?batches/s]

...

Validation Loss : 4.634754
Validation Perplexity : 103.002540
Start epoch 30, learning rate 0.000250 
2


 10%|█         | 3/29 [00:00<00:00, 29.85batches/s]

Epoch : 30, Batch : 327 / 328, Loss : 3.858495, Perplexity : 47.393972, Time : 30.623644


100%|██████████| 29/29 [00:00<00:00, 31.04batches/s]

Validation Loss : 4.637422
Validation Perplexity : 103.277793
